# Load library

In [1]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

# SentenceTokenizer Class

In [2]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",'로써',
        '해당', '합니다']
    def url2sentences(self, url):                   # URL일 경우 Text 추출
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)   # 텍스트에서 문장별로 반환
        print(article.text)
        print('*'*120)

        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
            return sentences

    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
              nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
            if noun not in self.stopwords and len(noun) > 1]))
        return nouns


# GrapMatrix Class

In [3]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence

    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

# Rank Class

In [4]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

# TextRank Class

In [5]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)

        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])

        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords


# Main

In [6]:
url = input()
textrank = TextRank(url)
for row in textrank.summarize(5):
    print(row)
    print()
print('keywords :',textrank.keywords())

안녕하세요.

교육상담 전문 학습플래너 김유진입니다.

​

동물보건사 자격증이 개설되면서 동물병원에서 근무할 때에

꼭 필요한 요소로 적용될 예정이며, 그로 인해 수의테크니션을

희망하거나 애완동물 관련 사업쪽으로 취업하려는 분들께서

동물보건사 자격증에 대해서 많이들 알아보고계십니다.

​

동물보건 관련 전공으로 최소 전문대학교 이상의 학력을 소지하거나

실무경력이 있어야지만 국가고시를 응시할 수 있습니다.

2021년부터 수의사법이 개정되고 국가자격으로 시행되면서

무조건 자격이 있어야지만 수의테크니션으로 근무를 할 수 있다고 합니다.

​

학점은행제에는 애완동물관리전공 전문학사학위 과정이

개설되어있으며, 민간자격증이 아닌 확실한 학력을 취득할 수 있으며

수의테크니션 자격을 갖추실 수 있습니다. 무엇보다 수능성적도 필요없고

고등학교 졸업 이상의 학력만 있다면 학점은행제를 이용하실 수 있습니다.

​

고졸학력으로 전문학사학위를 취득하려면

총 80학점을 이수하셔야하고, 애완동물관리전공 전문학사를

취득하시려면 전공 45학점 + 교양 15학점을 필수로

충족하고 총 80학점을 이수하셔야 합니다.

전공 45학점 중에서도 전공필수 18학점을 이수하셔야

안정적으로 전문학사학위 취득이 가능합니다.

​

문제는 온라인강의로 모든 전공학점을 채우는 것이 쉽지 않아서

대체수단으로 자격증을 취득하셔야해서, 애완동물관리전공을

진행하는 학생들은 전공자격증인 "축산산업기사"자격증을 병행합니다.

축산산업기사는 학점은행제로 41학점을 보유하고있으면 시험 응시가 가능합니다.

축산산업기사 자격증 취득 시 전공필수학점으로 인정됩니다.

​

학점은행제는 일반대학과 다르게

학년제가 아닌 학점제로 진행되어서 일정학점을

취득하면 학위를 수여받을 수 있어서 학점대체 가능한

자격증,독학사,전적대학점을 활용해서 학습기간을 단축하여

단기간에 학위취득이 가능합니다.

​

애완동물관리 전공 전문학사학위 취득하면 동물보건사

자격조건이 갖추어지고, 학점은행제 온라인 과정으